In [1]:
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import yaml

from PIL import Image as im
from tqdm import tqdm

In [2]:
eval_size = (256, 256, 32)

LABEL_COLORS = np.array([
    (255, 255, 255), # None
    (70, 70, 70),    # Building
    (100, 40, 40),   # Fences
    (55, 90, 80),    # Other
    (255, 255, 0),   # Pedestrian
    (153, 153, 153), # Pole
    (0, 0, 255),  # Road
    (100, 100, 100),  # Ground
    (150, 10, 10),  # Sidewalk
    (0, 155, 0),  # Vegetation
    (255, 0, 0)     # Vehicle
]).astype(np.uint8)

learning_map = {
  0 : 0,     # "unlabeled"
  1 : 0,     # "outlier" mapped to "unlabeled" --------------------------mapped
  10: 1,     # "car"
  11: 2,     # "bicycle"
  13: 5,     # "bus" mapped to "other-vehicle" --------------------------mapped
  15: 3,     # "motorcycle"
  16: 5,     # "on-rails" mapped to "other-vehicle" ---------------------mapped
  18: 4,     # "truck"
  20: 5,     # "other-vehicle"
  30: 6,     # "person"
  31: 7,     # "bicyclist"
  32: 8,     # "motorcyclist"
  40: 9,     # "road"
  44: 10,    # "parking"
  48: 11,    # "sidewalk"
  49: 12,    # "other-ground"
  50: 13,    # "building"
  51: 14,    # "fence"
  52: 0,     # "other-structure" mapped to "unlabeled" ------------------mapped
  60: 9,     # "lane-marking" to "road" ---------------------------------mapped
  70: 15,    # "vegetation"
  71: 16,    # "trunk"
  72: 17,    # "terrain"
  80: 18,    # "pole"
  81: 19,    # "traffic-sign"
  99: 0,     # "other-object" to "unlabeled" ----------------------------mapped
  252: 1,    # "moving-car" to "car" ------------------------------------mapped
  253: 7,    # "moving-bicyclist" to "bicyclist" ------------------------mapped
  254: 6,    # "moving-person" to "person" ------------------------------mapped
  255: 8,    # "moving-motorcyclist" to "motorcyclist" ------------------mapped
  256: 5,    # "moving-on-rails" mapped to "other-vehicle" --------------mapped
  257: 5,    # "moving-bus" mapped to "other-vehicle" -------------------mapped
  258: 4,    # "moving-truck" to "truck" --------------------------------mapped
  259: 5    # "moving-other"-vehicle to "other-vehicle" ----------------mapped
}

kitti_to_carla = {
    0: 0,
    1: 10,
    2: 10,
    3: 10,
    4: 10,
    5: 10,
    6: 4,
    7: 4,
    8: 4,
    9: 6,
    10: 6,
    11: 8,
    12: 7,
    13: 1,
    14: 9,
    15: 9,
    16: 7,
    17: 5,
    18: 5,
    19: 5
}

In [3]:
LABELS_REMAP = {}
for i in learning_map.keys():
    LABELS_REMAP[i] = kitti_to_carla[learning_map[i]]
print(LABELS_REMAP)

{0: 0, 1: 0, 10: 10, 11: 10, 13: 10, 15: 10, 16: 10, 18: 10, 20: 10, 30: 4, 31: 4, 32: 4, 40: 6, 44: 6, 48: 8, 49: 7, 50: 1, 51: 9, 52: 0, 60: 6, 70: 9, 71: 7, 72: 5, 80: 5, 81: 5, 99: 0, 252: 10, 253: 4, 254: 4, 255: 4, 256: 10, 257: 10, 258: 10, 259: 10}


In [4]:
sequence_path = "/home/tigeriv/Data/KITTI/dataset/sequences/08"
file_path = os.path.join(sequence_path, "voxels")

print("path:", file_path)

VISUALIZE = True
SAVE = True
TOP = True # get the top most label if true, otherwise get the most occuring labels

path: /home/tigeriv/Data/KITTI/dataset/sequences/08/voxels


In [5]:
def unpack(compressed):
    ''' given a bit encoded voxel grid, make a normal voxel grid out of it.  '''
    uncompressed = np.zeros(compressed.shape[0] * 8, dtype=np.uint8)
    uncompressed[::8] = compressed[:] >> 7 & 1
    uncompressed[1::8] = compressed[:] >> 6 & 1
    uncompressed[2::8] = compressed[:] >> 5 & 1
    uncompressed[3::8] = compressed[:] >> 4 & 1
    uncompressed[4::8] = compressed[:] >> 3 & 1
    uncompressed[5::8] = compressed[:] >> 2 & 1
    uncompressed[6::8] = compressed[:] >> 1 & 1
    uncompressed[7::8] = compressed[:] & 1
    return uncompressed

# Form BEV
def form_bev(labels, counts = None):
    bev_map = np.zeros((eval_size[0], eval_size[1]))
    bev_mask = np.sum(labels,axis=-1) != 0
    bev_x,bev_y = np.where(bev_mask)
    for i in range(bev_x.shape[0]):
        x = bev_x[i]
        y = bev_y[i]
        if TOP:
                lables_column = labels[x,y]
                
                mask_zero = lables_column != 0
                mask = np.sum(mask_zero) >= 1
                
                if mask:
                    indx = np.where(mask_zero)[0][-1]
                    label_xy = labels[x,y,indx]
                    remapped_xy = LABELS_REMAP[label_xy]
                    bev_map[x, y] = remapped_xy
            
        else:
            lables_column = labels[x,y]
            max_labels = np.unique(lables_column)
            if max_labels.shape[0] == 1 and max_labels[0] == 0:
                label_xy = 0
                remapped_xy = LABELS_REMAP[label_xy]
            else:
                label_xy = max_labels[1]
                remapped_xy = LABELS_REMAP[label_xy]
            
            bev_map[x, y] = remapped_xy
    return bev_map.astype(np.uint8)

def plot_bev(bev_map):
    plt.figure()
    ax = sns.heatmap(bev_map, cmap="YlGnBu", xticklabels=False, yticklabels=False)
    ax.invert_yaxis()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    plt.show()
    
def bev_img(bev_map):
    colored_data = LABEL_COLORS[bev_map.astype(int)]
    img = im.fromarray(colored_data, 'RGB')
    return img

In [6]:
if SAVE:
    bev_path = os.path.join(sequence_path, "bev_gt")
    if not os.path.exists(bev_path):
        os.makedirs(bev_path)
    total_files = int(len(os.listdir(file_path))/4)
    for i in tqdm(range(total_files)):
        labels_file = os.path.join(file_path, str(i).zfill(6) + ".label")
        labels = np.fromfile(labels_file,dtype=np.uint16).reshape(eval_size)
        bev_map = form_bev(labels)
        img = bev_img(bev_map)
        save_path = os.path.join(bev_path, str(i) + ".jpg")
        img.save(save_path)

 58%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                              | 1072/1837 [06:43<04:47,  2.66it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/tigeriv/Data/KITTI/dataset/sequences/08/voxels/001072.label'